In [ ]:
# default_exp ads

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
!pip install -qU nbdev fastcore

     |████████████████████████████████| 61kB 1.9MB/s 


In [ ]:
#hide
!pip install -qU gspread facebook_business python-facebook-api

     |████████████████████████████████| 1.0MB 3.8MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 10.1MB 13.9MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 


In [ ]:
#hide
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


In [ ]:
#hide
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

## Facebook Auth

In [ ]:
#hide
%%capture

import yaml
with open(path.parent/"facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env AD_ACC_ID {data.get("AD_ACC_ID")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
from tomorrowswig_followers.core import *
from tomorrowswig_followers.followers import get_new_followers

In [ ]:
#export
import os
from typing import *

from datetime import datetime
from functools import partial

import pandas as pd
import numpy as np
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adaccount import AdAccount

AD_ACC_ID = os.environ.get("AD_ACC_ID")

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Ads reporting

In [ ]:
#export
FacebookAdsApi.init(APP_ID, APP_SECRET, TOKEN)
account = AdAccount(AD_ACC_ID)

In [ ]:
datetime.fromtimestamp(1595946718)

datetime.datetime(2020, 7, 28, 14, 31, 58)

In [ ]:
datetime(2020,7,20,0,0).strftime('%s')

'1595203200'

In [ ]:
insights = account.get_insights(fields=[
    "ad_id",
    "ad_name",
    "clicks",
    "cpc",
    "ctr",
    "cpm",
    "cost_per_action_type",
    "spend",
    "impressions",
    "reach",
    "actions",
    "video_avg_time_watched_actions",
    "video_p50_watched_actions",
    "video_p75_watched_actions",
    "video_p95_watched_actions",

], params={
    'level': "ad",
    'date_preset': "yesterday",
    # "date_start": "1595116800",
    # "date_stop": "1595203200"
})


In [ ]:
from facebook_business.adobjects.adsinsights import *
fields = [f for f in AdsInsights.Field.__dict__ if not f.startswith("__")]

In [ ]:
" ".join([fields[56], fields[105]])

'cost_per_unique_conversion unique_conversions'

In [ ]:
# account.get_insights(fields=["ad_id", "quality_ranking", "engagement_rate_ranking",
#                              "conversion_rate_ranking", "ad_name"], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.yesterday,
# })

In [ ]:
# account.get_insights(fields=fields[:56] + fields[57:105] + fields[106:], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.last_3d,
# })[2]

In [ ]:
#export
def get_dif(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    return change_df

In [ ]:
#export
COLUMNS = ["Ad Name", "Ads Followers Change", "% (Clicks)", "% (Impressions)", "Cost Per Follow",
           "Clicks (All)", "Link Clicks", "CPC (All)" , "CPC (Cost per Link Click)",
           "CTR (All)", "CPM (Cost per 1,000 Impressions)", "Amount Spent (USD)", "Impressions", "Reach", "Post Reactions", "Post Shares",
           "Video Average Play Time", "Video Plays at 50%", "Video Plays at 75%", "Video Plays at 95%"]

In [ ]:
#export
get_insights = partial(account.get_insights, fields=[
        "ad_id",
        "ad_name",
        "clicks",
        "cpc",
        "ctr",
        "cpm",
        "cost_per_action_type",
        "spend",
        "impressions",
        "reach",
        "actions",
        "video_avg_time_watched_actions",
        "video_p50_watched_actions",
        "video_p75_watched_actions",
        "video_p95_watched_actions",
    ], params={
        'level': "ad",
        'date_preset': "yesterday",
    })

In [ ]:
#export
def get_action(cell, name):
    action = [a for a in cell if a["action_type"] == name]
    if action:
        return float(action[0]["value"])
    return 0

In [ ]:
#export
def get_followers_change(date: datetime) -> pd.DataFrame:
    history_df = get_df("History")
    new_followers = get_dif(history_df)
    followers_date = date.strftime("%b %d %Y")
    new_followers = new_followers.iloc[:,new_followers.columns.str.startswith(followers_date)].replace(0, np.nan)
    new_followers = new_followers.dropna(axis=1, how="all").iloc[:, :1].dropna().astype(int)
    if not new_followers.values.size:
        new_followers["no change"] = 0
    return new_followers

In [ ]:
assert get_followers_change(datetime.strptime("2020-07-25", "%Y-%m-%d")).columns[0] == "Jul 25 2020                 2020-07-25 19:10:35"
assert get_followers_change(datetime.strptime("2031-07-25", "%Y-%m-%d")).sum().item() == 0

In [ ]:
#export
def add_country_total(df: pd.DataFrame) -> Tuple[pd.DataFrame, str]:
    dupes = df[df.index.duplicated(False)]
    if not len(dupes):
        return

    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)"]
    to_sum = set(df.columns).difference(set(to_mean + ["ad_name"]))
    total = dupes.iloc[0,:].copy()
    index = dupes.index[0]
    total["ad_name"] = f"{index} total"
    total[to_mean] = dupes[to_mean].apply(np.mean)
    total[to_sum] = dupes[to_sum].astype(int).apply(sum)
    return df.append(total), index

In [ ]:
#export
def add_followers(df: pd.DataFrame, date: datetime):
    new_followers = get_followers_change(date)
    df["Ads Followers Change"] = new_followers
    df.fillna(0, inplace=True)
    df["% (Clicks)"] = (df["Ads Followers Change"] / df["clicks"])
    df["% (Impressions)"] = (df["Ads Followers Change"] / df["impressions"])
    df["Cost Per Follow"] = (df["spend"] / df["Ads Followers Change"])
    df.replace([np.inf], 0, inplace=True)

In [ ]:
ad = Ad("23844695610150691")

In [ ]:
ad = ad.api_get(fields=["targeting"])
ad["targeting"]["geo_locations"]["countries"][0]

In [ ]:
#export
def get_countries(ids: List[str]) -> List[Dict]:
    countries = []
    for id in ids:
        ad = Ad(id).api_get(fields=["targeting"])
        country = ad["targeting"]["geo_locations"]["countries"][0]
        countries.append({"ad_id": id, "country": country})
    return countries

In [ ]:
assert get_countries(["23844695610150691", "23844698190420691"]) == [{'ad_id': '23844695610150691', 'country': 'CA'},
                                                                     {'ad_id': '23844698190420691', 'country': 'US'}]

In [ ]:
def create_first_part():
    df = pd.DataFrame(get_insights())
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)

    df = df.merge(pd.DataFrame(get_countries(df["ad_id"].values)))
    df.set_index(df["country"], inplace=True)
    df.sort_values("ad_name", inplace=True)

    date = datetime.strptime(df["date_start"].values[0], "%Y-%m-%d")

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click"))
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                     "video_avg_time_watched_actions", "ad_id", "country"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    
    dupes = add_country_total(df)
    add_followers(df, date)
    return df.round(2), date

In [ ]:
df, date = create_first_part()

In [ ]:
df

,ad_name,clicks,cpc,cpm,ctr,impressions,reach,spend,Post Reactions,Post Shares,Link Clicks,CPC (Cost per Link Click),Video Average Play Time,Video Plays at 50%,Video Plays at 75%,Video Plays at 95%,Ads Followers Change,% (Clicks),% (Impressions),Cost Per Follow
country,,,,,,,,,,,,,,,,,,,,
AU,1 - Australia - General Ad 2-D,80,0.09,2.11,2.24,3578,3566,7.54,2.0,1.0,41.0,0.18,3.0,810.0,345.0,217.0,12,0.15,0.0,0.63
CA,1 - Canada - Gen #2 - D,191,0.10,2.17,2.12,9014,9014,19.60,9.0,2.0,101.0,0.19,3.0,1880.0,1017.0,630.0,-2,-0.01,-0.0,-9.80
GB,1 - UK - General Ad 2-D,244,0.12,2.30,1.99,12280,11957,28.22,3.0,1.0,138.0,0.20,3.0,2803.0,1290.0,727.0,20,0.08,0.0,1.41
US,1 - USA - Gen 2d - STORIES+FEED,451,0.16,2.89,1.84,24552,24400,70.85,68.0,6.0,224.0,0.32,4.0,5023.0,2899.0,1858.0,41,0.09,0.0,1.73
DE,2 - Germany - General Ad 2-D,206,0.08,1.91,2.43,8481,8481,16.22,6.0,3.0,109.0,0.15,3.0,2014.0,994.0,642.0,1,0.00,0.0,16.22
IT,2 - Italy - General Ad 2-D,204,0.07,0.58,0.83,24630,23592,14.29,9.0,1.0,90.0,0.16,2.0,3329.0,1303.0,712.0,10,0.05,0.0,1.43
SG,2 - Singapore - General Ad 2-D,133,0.07,1.45,2.02,6592,6508,9.57,3.0,0.0,69.0,0.14,2.0,1279.0,435.0,235.0,7,0.05,0.0,1.37
BR,3 - Brasil - General Ad 2-D,150,0.04,0.30,0.83,18036,18036,5.44,9.0,1.0,66.0,0.08,2.0,2636.0,837.0,515.0,39,0.26,0.0,0.14
ID,3 - Indonesia - General Ad 2-D,37,0.03,0.19,0.68,5423,5423,1.03,5.0,1.0,12.0,0.09,2.0,440.0,233.0,126.0,-3,-0.08,-0.0,-0.34


In [ ]:
to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)", "Video Average Play Time"]
empty = ["Ads Followers Change", "% (Clicks)", "% (Impressions)", "Cost Per Follow", "ad_name"]
to_sum = df.columns.difference(to_mean + empty)
total = pd.Series(name="total", index=df.columns, dtype=float)

In [ ]:
total[to_mean] = df[to_mean].apply(np.mean)
total[to_sum] = df[to_sum].apply(sum)

In [ ]:
df = df.append(total)

In [ ]:
pd.concat([df.iloc[-1:], df.iloc[:-1]])

,ad_name,clicks,cpc,cpm,ctr,impressions,reach,spend,Post Reactions,Post Shares,Link Clicks,CPC (Cost per Link Click),Video Average Play Time,Video Plays at 50%,Video Plays at 75%,Video Plays at 95%,Ads Followers Change,% (Clicks),% (Impressions),Cost Per Follow
country,,,,,,,,,,,,,,,,,,,,
total,NaN,1722.0,0.078,1.435,1.729,113710.0,112101.0,173.27,114.0,16.0,863.0,0.155,2.6,20366.0,9442.0,5706.0,NaN,NaN,NaN,NaN
AU,1 - Australia - General Ad 2-D,80.0,0.090,2.110,2.240,3578.0,3566.0,7.54,2.0,1.0,41.0,0.180,3.0,810.0,345.0,217.0,12.0,0.15,0.0,0.63
CA,1 - Canada - Gen #2 - D,191.0,0.100,2.170,2.120,9014.0,9014.0,19.60,9.0,2.0,101.0,0.190,3.0,1880.0,1017.0,630.0,-2.0,-0.01,-0.0,-9.80
GB,1 - UK - General Ad 2-D,244.0,0.120,2.300,1.990,12280.0,11957.0,28.22,3.0,1.0,138.0,0.200,3.0,2803.0,1290.0,727.0,20.0,0.08,0.0,1.41
US,1 - USA - Gen 2d - STORIES+FEED,451.0,0.160,2.890,1.840,24552.0,24400.0,70.85,68.0,6.0,224.0,0.320,4.0,5023.0,2899.0,1858.0,41.0,0.09,0.0,1.73
DE,2 - Germany - General Ad 2-D,206.0,0.080,1.910,2.430,8481.0,8481.0,16.22,6.0,3.0,109.0,0.150,3.0,2014.0,994.0,642.0,1.0,0.00,0.0,16.22
IT,2 - Italy - General Ad 2-D,204.0,0.070,0.580,0.830,24630.0,23592.0,14.29,9.0,1.0,90.0,0.160,2.0,3329.0,1303.0,712.0,10.0,0.05,0.0,1.43
SG,2 - Singapore - General Ad 2-D,133.0,0.070,1.450,2.020,6592.0,6508.0,9.57,3.0,0.0,69.0,0.140,2.0,1279.0,435.0,235.0,7.0,0.05,0.0,1.37
BR,3 - Brasil - General Ad 2-D,150.0,0.040,0.300,0.830,18036.0,18036.0,5.44,9.0,1.0,66.0,0.080,2.0,2636.0,837.0,515.0,39.0,0.26,0.0,0.14


In [ ]:
#export
def add_total(df: pd.DataFrame) -> pd.DataFrame:
    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)", "Video Average Play Time"]
    empty = ["Ads Followers Change", "% (Clicks)", "% (Impressions)", "Cost Per Follow", "ad_name"]
    to_sum = df.columns.difference(to_mean + empty)
    total = pd.Series(name="total", index=df.columns, dtype=float)
    total[to_mean] = df[to_mean].apply(np.mean)
    total[to_sum] = df[to_sum].apply(sum)
    df = df.append(total)
    return pd.concat([df.iloc[-1:], df.iloc[:-1]])

In [ ]:
#export
def get_insights_df(insights: List) -> Tuple[pd.DataFrame, datetime]:
    df = pd.DataFrame(insights)
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)

    df = df.merge(pd.DataFrame(get_countries(df["ad_id"].values)))
    df.set_index(df["country"], inplace=True)
    df.sort_values("ad_name", inplace=True)
    date = datetime.strptime(df["date_start"].values[0], "%Y-%m-%d")

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click"))
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                     "video_avg_time_watched_actions", "ad_id", "country"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    
    dupes = add_country_total(df)
    add_followers(df, date)
    df = add_total(df)
    df.rename(columns={"ad_name": "Ad Name", "clicks": "Clicks (All)", "cpc": "CPC (All)",
                       "ctr": "CTR (All)", "cpm": "CPM (Cost per 1,000 Impressions)",
                       "reach": "Reach", "impressions": "Impressions", "spend": "Amount Spent (USD)"}, inplace=True)
    df["% (Impressions)"] = df["% (Impressions)"].round(4)
    df[df.columns.difference(["% (Impressions)"])] = df[df.columns.difference(["% (Impressions)"])].round(2)
    return df[COLUMNS], date

In [ ]:
insights = get_insights()
df, date = get_insights_df(insights)

In [ ]:
df

,Ad Name,Ads Followers Change,% (Clicks),% (Impressions),Cost Per Follow,Clicks (All),Link Clicks,CPC (All),CPC (Cost per Link Click),CTR (All),"CPM (Cost per 1,000 Impressions)",Amount Spent (USD),Impressions,Reach,Post Reactions,Post Shares,Video Average Play Time,Video Plays at 50%,Video Plays at 75%,Video Plays at 95%
country,,,,,,,,,,,,,,,,,,,,
total,NaN,NaN,NaN,NaN,NaN,1722.0,863.0,0.08,0.16,1.73,1.44,173.27,113710.0,112101.0,114.0,16.0,2.6,20366.0,9442.0,5706.0
AU,1 - Australia - General Ad 2-D,12.0,0.15,0.0034,0.63,80.0,41.0,0.09,0.18,2.24,2.11,7.54,3578.0,3566.0,2.0,1.0,3.0,810.0,345.0,217.0
CA,1 - Canada - Gen #2 - D,-2.0,-0.01,-0.0002,-9.80,191.0,101.0,0.10,0.19,2.12,2.17,19.60,9014.0,9014.0,9.0,2.0,3.0,1880.0,1017.0,630.0
GB,1 - UK - General Ad 2-D,20.0,0.08,0.0016,1.41,244.0,138.0,0.12,0.20,1.99,2.30,28.22,12280.0,11957.0,3.0,1.0,3.0,2803.0,1290.0,727.0
US,1 - USA - Gen 2d - STORIES+FEED,41.0,0.09,0.0017,1.73,451.0,224.0,0.16,0.32,1.84,2.89,70.85,24552.0,24400.0,68.0,6.0,4.0,5023.0,2899.0,1858.0
DE,2 - Germany - General Ad 2-D,1.0,0.00,0.0001,16.22,206.0,109.0,0.08,0.15,2.43,1.91,16.22,8481.0,8481.0,6.0,3.0,3.0,2014.0,994.0,642.0
IT,2 - Italy - General Ad 2-D,10.0,0.05,0.0004,1.43,204.0,90.0,0.07,0.16,0.83,0.58,14.29,24630.0,23592.0,9.0,1.0,2.0,3329.0,1303.0,712.0
SG,2 - Singapore - General Ad 2-D,7.0,0.05,0.0011,1.37,133.0,69.0,0.07,0.14,2.02,1.45,9.57,6592.0,6508.0,3.0,0.0,2.0,1279.0,435.0,235.0
BR,3 - Brasil - General Ad 2-D,39.0,0.26,0.0022,0.14,150.0,66.0,0.04,0.08,0.83,0.30,5.44,18036.0,18036.0,9.0,1.0,2.0,2636.0,837.0,515.0


In [ ]:
stats = pd.Series(dtype=float, name="Followers")
ads_followers_gain = df["Ads Followers Change"].sum()
followers_gain = get_followers_change(date).sum().item()
spent = df.loc["total", "Amount Spent (USD)"]

In [ ]:
stats["Ads Followers Change"] = ads_followers_gain
stats["New Followers"] = get_new_followers()[1]
stats["Random Followers"] = stats["New Followers"] - ads_followers_gain
stats["Unfollowers"] = stats["New Followers"] - followers_gain
stats["Followers Change"] = ads_followers_gain + stats["Random Followers"] - stats["Unfollowers"]

In [ ]:
stats_df = pd.DataFrame(stats)
stats_df.insert(0, "  ", stats_df.index)

In [ ]:
stats_df["Conversion Rate"] = stats_df["Followers"] / df.loc["total", "Clicks (All)"]
stats_df["Cost Per Follower/Increase"] = spent/ stats_df["Followers"]
stats_df.iloc[2:5, 2] = 0
stats_df.iloc[2:4, 3] = 0

random followers + followers from ads - unfollowers = followers gain

unfollowers = random followers + followers from ads - followers gain = new followers - followers gain

random_followers = new followers - followers from ads

In [ ]:
#export
def more_stats(df: pd.DataFrame, date: datetime) -> pd.DataFrame:
    stats = pd.Series(dtype=float, name="Followers")
    ads_followers_gain = df["Ads Followers Change"].sum()
    followers_gain = get_followers_change(date).sum().item()
    spent = df.loc["total", "Amount Spent (USD)"]
    stats["Ads Followers Change"] = ads_followers_gain
    new_followers = get_new_followers()[1]
    stats["Random Followers"] = new_followers - ads_followers_gain
    stats["New Followers"] = new_followers
    stats["Unfollowers"] = stats["New Followers"] - followers_gain
    stats["Followers Change"] = ads_followers_gain + stats["Random Followers"] - stats["Unfollowers"]

    stats_df = pd.DataFrame(stats)
    stats_df.index.name = "  "
    stats_df["Conversion Rate"] = stats_df["Followers"] / df.loc["total", "Clicks (All)"]
    stats_df[" "] = 0
    stats_df["Cost Per Follower/Increase"] = spent/ stats_df["Followers"]
    stats_df.iloc[1:, 1] = 0
    stats_df.iloc[[1,3,4], 3] = 0
    return stats_df.round(2)

In [ ]:
#export
def create_insights(event: Dict = None, context=None,) -> str:
    insights = get_insights()
    df, date = get_insights_df(insights)
    worksheet_name = date.strftime("%b %d %Y")
    stats_df = more_stats(df, date)
    empty = pd.Series(name="", dtype=str)
    df = df.append([empty] * 4)
    write_df(df, worksheet_name)
    stats_df = stats_df.append([empty] * 15)
    write_df(stats_df, worksheet_name, loc=f"B{len(df)}")
    notes_df = pd.DataFrame([["", "CONSIDER:", "", "", "TO DO:"]], index=["PERFORMANCE:"], columns=[""] * 5)
    write_df(notes_df, worksheet_name, loc=f"A{len(stats_df) + 1}")

    wsh = get_worksheet(worksheet_name)
    wsh.format(f"C1:G{len(df)-1}", {"textFormat": {
        "foregroundColor": {
            "red": 0.45,
            "green": 0.0,
            "blue": 0.0
        }}},)
    wsh.format(f"D:E", {"numberFormat": {"type": "PERCENT"}},)
    wsh.format(f"F", {"numberFormat": {"type": "CURRENCY"}},)

    unf_cell = wsh.find("Unfollowers")
    wsh.format(f"C{wsh.find('Unfollowers').row}", {"textFormat": {
        "foregroundColor": {
            "red": 0.0,
            "green": 0.0,
            "blue": 1.0
        }}},)
    return f"Created insights in '{worksheet_name}'"

In [ ]:
create_insights()

"Created insights in 'Aug 09 2020'"

In [ ]:
# DONE estimate, real

# SHARE
## check all fields data

# TODO

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
